# Tokenizacion #
Para el primer paso, tokenizaremos los titulos de youtobe.

In [2]:
import nltk
import pandas as pd

nltk.download('punkt')
nltk.download('punkt_tab')

# Cargar el archivo CSV
data = pd.read_csv('../data/MXvideos.csv', nrows=40000, errors='ignore')  # Cargar las primeras 5000 filas
allReviews = data['title']  # Extraer la columna de texto

tokesReview ={}#arreglo de tokens

for iterador, review in enumerate(allReviews):
    
    # Tokenizar la reseña
    tokens = nltk.word_tokenize(review)
    video_id = data.iloc[iterador]['video_id']

    # Asignar los tokens a su 'review_id' correspondiente
    tokesReview[video_id] = tokens

    # Incrementamos el iterador
    iterador += 1

[nltk_data] Downloading package punkt to C:\Users\MARTHA ISABEL
[nltk_data]     VILLAM\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\MARTHA ISABEL
[nltk_data]     VILLAM\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


TypeError: read_csv() got an unexpected keyword argument 'errors'